In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'GeForce RTX 3090'

In [5]:
SEED=1

In [6]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [7]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [9]:
##########################
ARCH = 'ResNet-18'
########################
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2
TRAINDIR="./train"
VALDIR="./val"

In [ ]:
# TRAINDIR="/CINIC/train"
# VALDIR="/CINIC/valid"

### Check if cuda is available here

In [10]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
torch.cuda.is_available()

True

### Assign your GPU below

In [11]:
# Assign your GPU in this cell
GPU = torch.cuda.current_device()

In [12]:
# set your active device to your GPU in this cell
torch.cuda.set_device(GPU)

In [13]:
# enable algorithm optimization
cudnn.benchmark = True

In [14]:
torch.autograd.set_detect_anomaly(True)

### Fill in the heart of the train section below

In [15]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        images = images.cuda(GPU, non_blocking=True)
        target = target.cuda(GPU, non_blocking=True)

        # compute output
        # output = model ?? images
        output = model(images)

        # compute loss 
        # loss = criterion, output, target
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        ## optimizer ..??
        optimizer.zero_grad()

        ## backprop!
        ### loss... ???
        loss.backward()
        
        # update the weights!
        ## optimier .. ??
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [16]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    # model ???
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            images, target = images.to(GPU), target.to(GPU)

            # compute output
            # output = model ??? images?
            output = model(images)

            # compute loss
            # loss  = criterion ?? output ?? target
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [17]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [18]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [19]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [20]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [21]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [22]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [23]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [24]:
#IMG_SIZE = 32
IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [25]:
# select the model
# model = ... 
model = models.resnet18()

### Send the model to the cuda device

In [26]:
# send the model to the cuda device.. 
model = model.to(GPU)

### Instantiate the loss to cross entropy

In [27]:
# use the cross-entropy loss
criterion = torch.nn.CrossEntropyLoss()

### Instantiate the optimizer to SGD

In [28]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

#### Create the learning rate scheduler

In [29]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=10)

In [32]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

### Create the train dataset object

In [33]:
# use torchvision.datasets.CIFAR10
train_dataset = datasets.ImageFolder(TRAINDIR, transform_train)

In [40]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

### Create the val dataset object

In [41]:
# use torchvision.datasets.CIFAR10
val_dataset = datasets.ImageFolder(VALDIR, transform_val)

### Create the train dataloader

In [42]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH, shuffle=True, num_workers=WORKERS)

### Create the c

In [43]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=VAL_BATCH, shuffle=True, num_workers=WORKERS) 

In [44]:
best_acc1 = 0

In [45]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][    0/10010]	Time  9.151 ( 9.151)	Data  8.789 ( 8.789)	Loss 3.9196e+00 (3.9196e+00)	Acc@1  27.34 ( 27.34)	Acc@5  45.31 ( 45.31)
Epoch: [0][   50/10010]	Time  1.016 ( 0.831)	Data  0.773 ( 0.583)	Loss 3.7073e+00 (3.7584e+00)	Acc@1  25.00 ( 24.42)	Acc@5  46.88 ( 46.80)
Epoch: [0][  100/10010]	Time  0.229 ( 0.764)	Data  0.013 ( 0.483)	Loss 3.5787e+00 (3.7502e+00)	Acc@1  26.56 ( 24.31)	Acc@5  48.44 ( 47.15)
Epoch: [0][  150/10010]	Time  0.970 ( 0.823)	Data  0.013 ( 0.352)	Loss 3.7858e+00 (3.7166e+00)	Acc@1  22.66 ( 25.11)	Acc@5  48.44 ( 48.03)
Epoch: [0][  200/10010]	Time  0.988 ( 0.863)	Data  0.013 ( 0.268)	Loss 3.6046e+00 (3.7034e+00)	Acc@1  27.34 ( 25.32)	Acc@5  46.09 ( 48.24)
Epoch: [0][  250/10010]	Time  0.985 ( 0.887)	Data  0.013 ( 0.217)	Loss 3.6002e+00 (3.6993e+00)	Acc@1  26.56 ( 25.45)	Acc@5  51.56 ( 48.45)
Epoch: [0][  300/10010]	Time  1.002 ( 0.900)	Data  0.013 ( 0.184)	Loss 3.2482e+00 (3.6941e+00)	Acc@1  32.03 ( 25.41)	Acc@5  61.72 ( 48.51)
Epoch: [0][  350/10010]	Tim

KeyboardInterrupt: 